# Imports

In [9]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

import logging
import traceback
from pprint import pformat

from geopy.distance import geodesic

## Parameters

In [2]:
creds_path:str = "./keys/serviceAccountKey.json"
max_distance_km:float = 5.0


## Setup

In [3]:
logger = logging.getLogger('my_logger')

logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s: %(asctime)s  - %(message)s')
console_handler.setFormatter(formatter)

logger.addHandler(console_handler)

## Authentication

In [4]:
try:
    cred = credentials.Certificate(creds_path)
    logger.info('Firebase centantial read successfully')
    firebase_admin.initialize_app(cred)
    logger.info('Firebase initialized successfully')
    db = firestore.client()
    logger.info('Firestore client created')

except Exception as e:
    traceback_str = traceback.format_exc()
    logger.error('An error occurred: %s', str(e))
    logger.debug(f'Traceback: {traceback_str}')

INFO: 2024-01-19 02:22:26,086  - Firebase centantial read successfully
INFO: 2024-01-19 02:22:26,087  - Firebase initialized successfully
INFO: 2024-01-19 02:22:26,088  - Firestore client created


## Utils

In [5]:
def getAllMarkers(firebase_clent):
    markers = []
    
    try:
        markers_get = firebase_clent.collection('Markers').get()
        logger.info('Successfully retrived all Markers')
        
    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error('An error occurred: %s', str(e))
        logger.debug(f'Traceback: {traceback_str}')
        
    for m in markers_get:
        marker = {
            "marker-id":m.to_dict()["id"],
            "marker_cord":(m.to_dict()["lat"],m.to_dict()["long"]),
            }
        markers.append(marker)
        
    logger.info('Successfully got all Markers into python dict')
    return markers

In [13]:
def find_markers_within_distance(
    markers:list, 
    target_coord:tuple, 
    max_distance_km:float=5.0
    ):

    nearby_markers = []

    try:
        for marker in markers:
            marker_coord = marker['marker_cord']
            distance = geodesic(target_coord, marker_coord).kilometers

            if distance <= max_distance_km:
                nearby_markers.append(marker)
        logger.debug("Successfully looped over all markers and selected markers within distance")
    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error('An error occurred: %s', str(e))
        logger.debug(f'Traceback: {traceback_str}')
    
    logger.info(f"Successfully got nearby markers: {len(nearby_markers)}") 
    return nearby_markers

## Program

In [7]:
markers_data = getAllMarkers(db)
logger.info(f"Total markers retrived {len(markers_data)}")
logger.debug(pformat(markers_data))

INFO: 2024-01-19 02:24:14,531  - Toatl markers retrived33


In [15]:
nearby_markers = find_markers_within_distance(
    markers_data, 
    (26.52625454590057, 93.88349554741812)
    )

INFO: 2024-01-19 02:37:23,164  - Successfully got nearby markers: 4


In [22]:
from sklearn.cluster import DBSCAN
import numpy as np
# Extract coordinates from markers
coordinates = np.array([marker['marker_cord'] for marker in markers_data])

# Specify DBSCAN parameters
epsilon = 1.0  # Maximum distance between two samples for one to be considered as in the neighborhood of the other
min_samples = 2  # The number of samples (or total weight) in a neighborhood for a point to be considered as a core point

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
labels = dbscan.fit_predict(coordinates)

# Add cluster labels to the markers
for i, marker in enumerate(markers_data):
    marker['cluster_label'] = labels[i]

# Display the clusters
clusters = set(labels)
for cluster in clusters:
    cluster_markers = [marker for marker in markers_data if marker['cluster_label'] == cluster]
    print(f"Cluster {cluster}: {cluster_markers}")
    print(f"Length: {len(cluster_markers)}")

Cluster 0: [{'marker-id': 1704701781090, 'marker_cord': (26.1422046, 91.6618141), 'cluster_label': 0}, {'marker-id': 1705343566607, 'marker_cord': (26.1362523933542, 91.72798164188862), 'cluster_label': 0}, {'marker-id': 1702966788552, 'marker_cord': (26.1060139519425, 91.59353524446487), 'cluster_label': 0}, {'marker-id': 1703486380661, 'marker_cord': (26.140288349728852, 91.76433496177197), 'cluster_label': 0}, {'marker-id': 1705075213364, 'marker_cord': (26.156115693417036, 91.67581502348185), 'cluster_label': 0}, {'marker-id': 1705263539822, 'marker_cord': (26.147598707001983, 91.6728488355875), 'cluster_label': 0}, {'marker-id': 1705075157599, 'marker_cord': (26.15511686065222, 91.67470794171095), 'cluster_label': 0}, {'marker-id': 1705075176799, 'marker_cord': (26.154326876889748, 91.67576909065247), 'cluster_label': 0}, {'marker-id': 1705344721585, 'marker_cord': (26.088155422482505, 91.53074339032173), 'cluster_label': 0}, {'marker-id': 1703667027697, 'marker_cord': (26.1988244

In [25]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(india_map)

# Add markers to the map
for marker in markers_data:
    folium.Marker(location=marker['marker_cord'], 
                  popup=f"Marker ID: {marker['marker-id']}, Cluster: {marker['cluster_label']}").add_to(marker_cluster)

# Save the map to an HTML file or display it
india_map

## class

In [ ]:
class NearYou:
    def __init__(self, firebase_clent, max_distance_km:float) -> None:
        self.getAllMarkers(firebase_clent)
        self.max_distance_km = max_distance_km
        pass
    def getAllMarkers(self, firebase_clent)-> None:
        markers = []
        
        try:
            markers_get = firebase_clent.collection('Markers').get()
            logger.info('Successfully retrived all Markers')
            
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error('An error occurred: %s', str(e))
            logger.debug(f'Traceback: {traceback_str}')
            
        for m in markers_get:
            marker = {
                "marker-id":m.to_dict()["id"],
                "marker_cord":(m.to_dict()["lat"],m.to_dict()["long"]),
                }
            markers.append(marker)
            
        logger.info('Successfully got all Markers into python dict')
        
        self.markers = markers
    def find_markers_within_distance(
        self,
        target_coord:tuple
        ):

        nearby_markers = []

        try:
            for marker in self.markers:
                marker_coord = marker['marker_cord']
                distance = geodesic(target_coord, marker_coord).kilometers

                if distance <= self.max_distance_km:
                    nearby_markers.append(marker)
            logger.debug("Successfully looped over all markers and selected markers within distance")
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error('An error occurred: %s', str(e))
            logger.debug(f'Traceback: {traceback_str}')

        return nearby_markers